In [1]:
from torchvision.datasets import Cityscapes
from torchvision import transforms
from torch.utils.data import DataLoader, Subset

image_size = (64, 128)
root_dir = "datasets/cityscapes"

input_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

target_transform = transforms.Compose([
    transforms.Resize(
        image_size,
        interpolation=transforms.InterpolationMode.NEAREST
    )
])


full_train_dataset = Cityscapes(
    root=root_dir,
    split='train',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

full_val_dataset = Cityscapes(
    root=root_dir,
    split='val',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

In [2]:
from utils.Cityscapes.CityscapesWrapper import CityscapesWrapper

train_samples = 800
val_samples = 200

train_subset = CityscapesWrapper(
    Subset(
        full_train_dataset,
        range(train_samples)
    ),
    target_transform=target_transform
)
val_subset = CityscapesWrapper(
    Subset(
        full_val_dataset,
        range(val_samples)
    ),
    target_transform=target_transform
)

train_loader = DataLoader(
    train_subset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)
val_loader = DataLoader(
    val_subset,
    batch_size=4,
    shuffle=False,
    num_workers=0
)

In [3]:
from utils.Cityscapes.UNetTorchCityscapes import UNet
import torch.optim as optim
import torch.nn as nn

device = "mps"
model = UNet(
    in_channels=3,
    out_channels=34
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-3
)

In [ ]:
from utils.Cityscapes.CityscapesTrainingUtils import train_one_epoch_cityscapes, validate_cityscapes

train_losses = []
val_losses = []

num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    train_loss = train_one_epoch_cityscapes(model, train_loader, optimizer, criterion)
    val_loss = validate_cityscapes(model, val_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")


Epoch 1/10


Validation: 100%|██████████| 50/50 [00:13<00:00,  3.84it/s]


Train Loss: 2.0081 | Val Loss: 1.1947

Epoch 2/10


Validation: 100%|██████████| 50/50 [00:13<00:00,  3.81it/s]


Train Loss: 1.1861 | Val Loss: 1.0645

Epoch 3/10


...

Validation: 100%|██████████| 50/50 [00:13<00:00,  3.83it/s]


Train Loss: 1.0178 | Val Loss: 0.9393

Epoch 5/10


Validation: 100%|██████████| 50/50 [00:13<00:00,  3.84it/s]


Train Loss: 0.9518 | Val Loss: 0.9309

Epoch 6/10


Training:  43%|████▎     | 86/200 [00:26<00:35,  3.18it/s]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, num_epochs + 1), val_losses, label='Val Loss', marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import torch

model.eval()
with torch.no_grad():
    sample_image, sample_target = next(iter(val_loader))
    sample_image = sample_image.to(device)
    sample_target = sample_target.to(device)

    output = model(sample_image)
    pred_mask = torch.argmax(output, dim=1)

img = sample_image[0].cpu().permute(1, 2, 0).numpy()
gt_mask = sample_target[0].cpu().squeeze().numpy()
pred_mask = pred_mask[0].cpu().numpy()

def apply_colormap(mask):
    colormap = np.array([
        [0, 0, 0],         # class 0: black
        [0, 255, 0],       # class 1: green
        [0, 0, 255],       # class 2: blue
        [255, 0, 0],       # class 3: red
        [255, 255, 0],     # etc.
    ])
    mask_rgb = colormap[mask % len(colormap)]
    return mask_rgb.astype(np.uint8)

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(img)
axs[0].set_title("Input Image")
axs[1].imshow(apply_colormap(gt_mask))
axs[1].set_title("Ground Truth Mask")
axs[2].imshow(apply_colormap(pred_mask))
axs[2].set_title("Predicted Mask")

for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), "./weights/cityscapes_unet.pth")

# Ensemble

In [ ]:
num_classes = 34

model1 = UNet(in_channels=3, out_channels=num_classes).to(device)
model2 = UNet(in_channels=3, out_channels=num_classes).to(device)
model3 = UNet(in_channels=3, out_channels=num_classes).to(device)

ensemble_models = [model1, model2, model3]

In [ ]:
optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-3)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=1e-3)

ensemble_optimizers = [optimizer1, optimizer2, optimizer3]

In [ ]:
from tqdm import tqdm

def train_one_epoch_ensemble(models, optimizers, loader, criterion):
    for model in models:
        model.train()

    total_loss = 0

    for images, targets in tqdm(loader, desc="Training"):
        images = images.to(device)
        targets = targets.to(device).long().squeeze(1)

        batch_loss = 0

        for model, optimizer in zip(models, optimizers):
            outputs = model(images)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss += loss.item()

        total_loss += batch_loss / len(models)  # Average loss over ensemble

    return total_loss / len(loader)


def validate_ensemble(models, loader, criterion):
    for model in models:
        model.eval()

    total_loss = 0

    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device)
            targets = targets.to(device).long().squeeze(1)

            # Get ensemble predictions (softmax average)
            outputs = [torch.softmax(model(images), dim=1) for model in models]
            avg_output = torch.stack(outputs).mean(dim=0)

            loss = criterion(avg_output, targets)
            total_loss += loss.item()

    return total_loss / len(loader)

In [ ]:
num_epochs = 5
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    train_loss = train_one_epoch_ensemble(ensemble_models, ensemble_optimizers, train_loader, criterion)
    val_loss = validate_ensemble(ensemble_models, val_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

In [ ]:
for i, model in enumerate(ensemble_models):
    torch.save(model.state_dict(), f"./weights/cityscapes_ensemble_unet_{i+1}.pth")